In [1]:
import pandas as pd
# Import CSV files as DataFrames
df1 = pd.read_csv('/Users/stella/Documents/tierney/project/combining datasets/cross_solvent_solubility_dataset_BSc_2024.csv')
df2 = pd.read_csv('/Users/stella/Documents/tierney/project/combining datasets/basf_inchi_logS.csv')

# Drop column 'MP_C' from df2
if 'MP_C' in df2.columns:
    df2 = df2.drop(columns=['MP_C'])

# Remove 'InChiKey=' from all entries in df2['solute_inchikey']
df2['solute_inchikey'] = df2['solute_inchikey'].str.replace('InChIKey=', '', regex=False)

# Combine df1 and df2, then remove duplicates
combined_df = pd.concat([df1, df2]).drop_duplicates()

# Display the first few rows of the merged DataFrame
print(combined_df.head())

# Save merged DataFrame to a CSV file
combined_df.to_csv('combined_dataset.csv', index=False)

       ID                                        solute_name  \
0  ID0001                                 (-)-levonorgestrel   
1  ID0002                                 (-)-levonorgestrel   
2  ID0003         (1-diethylamino-1-oxopropan-2-yl) benzoate   
3  ID0004        (1,1-dioxido-1,2-thiazinan-2-yl)acetic acid   
4  ID0005  (1,2-dibromo-2,2-dichloroethyl) dimethyl phosp...   

                                       solute_smiles solvent_name  \
0  O=C4\C=C3/[C@@H]([C@H]2CC[C@]1([C@@H](CC[C@]1(...    1-octanol   
1  O=C4\C=C3/[C@@H]([C@H]2CC[C@]1([C@@H](CC[C@]1(...    1-octanol   
2                   C(=O)(C(OC(=O)c1ccccc1)C)N(CC)CC        water   
3                          S1(=O)(=O)N(CC(=O)O)CCCC1        water   
4                       BrC(Cl)(Cl)C(Br)OP(=O)(OC)OC        water   

  solvent_smiles              solute_inchikey       S_M LogS_M       MW  
0      OCCCCCCCC  WWYNJERNGUHSAO-XUDSTZEENA-N  0.014791  -1.83  312.453  
1      OCCCCCCCC  WWYNJERNGUHSAO-XUDSTZEENA-N  0.014

In [8]:
import pandas as pd

df = pd.read_csv("combined_dataset.csv")
# Count unique entries
unique_solutes = df['solute_inchikey'].nunique()
unique_solvents = df['solvent_name'].nunique()

print(f"Number of unique solutes InChIKeys: {unique_solutes}")
print(f"Number of unique solvents names: {unique_solvents}")


Number of unique solutes InChIKeys: 3352
Number of unique solvents names: 189


In [9]:
import pandas as pd

# Load the dataset
file_path = "/Users/stella/Documents/tierney/project/combined datasets/combined_data_no_duplicates_and_ave_duplicates.csv"  # Replace with your file path
df = pd.read_csv(file_path)
print(f"Total rows:{df.shape[0]}")
exact_duplicates=df[df.duplicated(keep=False)]
print(f"total:{exact_duplicates.shape[0]}")
df.head()

Total rows:13405
total:0


,solvent_name,solvent_smiles,solute_inchikey,LogS,MW
0,water,O,BZZFPIRKNVWTKJ-UHFFFAOYNA-N,-2.09599998,249.310
1,water,O,HQKJYAGNQSMRJK-FZOZFQFYNA-N,-0.585000001,193.224
2,water,O,BUYMVQAILCEWRR-UHFFFAOYNA-N,-2.279999967,380.784
3,water,O,UFMJMAPQIWYKER-UHFFFAOYNA-N,-1.815999988,392.357
4,water,O,WSORODGWGUUOBO-UHFFFAOYNA-N,-3.335999599,314.296


In [ ]:
import pandas as pd

combined_df = pd.read_csv("/Users/stella/Documents/tierney/project/combined_data.csv")
# Filter out rows where solute_name contains "DONOTUSE"
filtered_combined_df = combined_df[~combined_df["solute_name"].str.contains("DONOTUSE", case=False, na=False)]
# Filter out rows where solvent_name contains a "/"
filtered_combined_df = combined_df[~combined_df["solvent_name"].str.contains("/", na=False)]
filtered_combined_df.to_csv("filtered_combined_data.csv", index=False)
print(f"Original dataset size: {len(combined_df)}")
print(f"Filtered dataset size: {len(filtered_combined_df)}")
print(f"Number of rows removed: {len(combined_df) - len(filtered_combined_df)}")

Original dataset size: 17930
Filtered dataset size: 17289
Number of rows removed: 641


In [4]:
#  Read in the CSV file
df = pd.read_csv("filtered_combined_data.csv")

# Remove specified columns
df = df.drop(columns=['solute_name', 'S_M', 'ID', 'MW'])

# Identify duplicate rows except for 'LogS' and 'ID'
columns_to_check = [col for col in df.columns if col not in ['LogS']]
duplicates = df[df.duplicated(subset=columns_to_check, keep=False)]

# Save duplicate rows
duplicates.to_csv('filtered_combined_dataset_duplicates.csv', index=False)

In [5]:
# Remove original duplicate rows from the dataframe
df_no_duplicates = df.drop(duplicates.index)
df_no_duplicates.to_csv('filtered_combined_dataset_no_duplicates.csv', index=False)

# Ensure 'LogS' is numeric before averaging
duplicates['LogS_M'] = pd.to_numeric(duplicates['LogS_M'], errors='coerce')

# Compute the average 'LogS' for duplicate groups
averaged_duplicates = duplicates.groupby(columns_to_check, as_index=False)['LogS_M'].mean()

# Append the averaged rows
df_no_duplicates = pd.concat([df_no_duplicates, averaged_duplicates], ignore_index=True)

# Save the final dataframe
df_no_duplicates.to_csv('final_combined_data.csv', index=False)

/var/folders/m2/ptj22ckn1lq9dwc6k6bwvz580000gn/T/ipykernel_38154/2322379026.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates['LogS_M'] = pd.to_numeric(duplicates['LogS_M'], errors='coerce')


In [6]:
import pandas as pd
import os

# Load the dataset
file_path = "/Users/stella/Documents/tierney/project/combined datasets/final_combined_data.csv"  # Replace with your file path

# Check if the file exists
if not os.path.exists(file_path):
	# Create the file from the existing dataframe
	df_no_duplicates.to_csv(file_path, index=False)

# Load the dataset
df = pd.read_csv(file_path)
print(f"Total rows: {df.shape[0]}")
exact_duplicates = df[df.duplicated(keep=False)]
print(f"total: {exact_duplicates.shape[0]}")
df.head()

Total rows: 9316
total: 0


,solute_smiles,solvent_name,solvent_smiles,solute_inchikey,LogS_M
0,FC(F)(F)c1cc(c(cc1)NC(C(C)C)C(=O)OC(c2cc(ccc2)...,water,O,INISTDXBRIBGOC-UHFFFAOYNA-N,-8.0
1,CC(Cc1cccc(c1)OC)NC(C)Cc2ccccc2OC,water,O,HSNNLWPVPBKGQW-UHFFFAOYNA-N,-3.0
2,OCCCC,water,O,LRHPLDYGYMQRHN-UHFFFAOYNA-N,0.0
3,C(CCC)O,water,O,LRHPLDYGYMQRHN-UHFFFAOYNA-N,0.0
4,ClCC(C)C,water,O,QTBFPMKWQKYFLR-UHFFFAOYNA-N,-2.0


In [9]:
import pandas as pd

# Load the dataset
file_path = "/Users/stella/Documents/tierney/project/notebooks/final_combined_data.csv"  # Replace with your file path
df = pd.read_csv(file_path)

# Print initial statistics
print(f"Total rows before processing: {df.shape[0]}")

# Identify duplicates based on solvent_name and solute_inchikey
duplicates = df[df.duplicated(subset=['solvent_name', 'solute_inchikey'], keep=False)]
print(f"Total duplicate rows based on solvent_name and solute_inchikey: {duplicates.shape[0]}")

# Ensure 'LogS_M' is numeric before averaging
df['LogS_M'] = pd.to_numeric(df['LogS_M'], errors='coerce')

# Group by solvent_name and solute_inchikey, calculate the mean of LogS_M, and keep the first row
df_cleaned = df.groupby(['solvent_name', 'solute_inchikey'], as_index=False).agg(
    {
        'LogS_M': 'mean',  # Average LogS_M for duplicates
        **{col: 'first' for col in df.columns if col not in ['LogS_M']}  # Keep the first value for all other columns
    }
)

# Print final statistics
print(f"Total rows after processing: {df_cleaned.shape[0]}")
print(f"Number of unique solvent-solute pairs: {df_cleaned[['solvent_name', 'solute_inchikey']].drop_duplicates().shape[0]}")

# Save the cleaned dataset to a new CSV file
output_path = "/Users/stella/Documents/tierney/project/notebooks/final_combined_data_cleaned.csv"
df_cleaned.to_csv(output_path, index=False)
print(f"Cleaned dataset saved to: {output_path}")

Total rows before processing: 9316
Total duplicate rows based on solvent_name and solute_inchikey: 3316
Total rows after processing: 7303
Number of unique solvent-solute pairs: 7303
Cleaned dataset saved to: /Users/stella/Documents/tierney/project/notebooks/final_combined_data_cleaned.csv


In [ ]:
import pandas as pd

# Load the dataset
file_path = "/Users/stella/Documents/tierney/project/notebooks/final_combined_data_cleaned.csv"  
df = pd.read_csv(file_path)
print(f"Total rows:{df.shape[0]}")
exact_duplicates=df[df.duplicated(keep=False)]
print(f"total:{exact_duplicates.shape[0]}")
df.head()

Total rows:7303
total:0


,LogS_M,solute_smiles,solvent_name,solvent_smiles,solute_inchikey
0,-1.698970,O=Cc2cc3c(c1c2cccc1)cccc3,"1,1,2-trichlorotrifluoroethane",ClC(F)(F)C(Cl)(Cl)F,QECIGCMPORCORE-UHFFFAOYNA-N
1,-0.160929,Fc2c1c(F)c(F)c(F)c(F)c1c(F)c(F)c2F,"1,2-dibromoethane",BrCCBr,JDCMOHAFGDQQJX-UHFFFAOYNA-N
2,-0.338571,Fc1c(c(F)c(F)c(F)c1F)c2c(F)c(F)c(F)c(F)c2F,"1,2-dibromoethane",BrCCBr,ONUFSRWQCKNVSL-UHFFFAOYNA-N
3,0.490035,c1ccc2ccccc2c1,"1,2-dibromoethane",BrCCBr,UFWIBTONFRDIAS-UHFFFAOYNA-N
4,0.220052,ClC(Cl)(Cl)C(Cl)(Cl)Cl,"1,2-dibromoethane",BrCCBr,VHHHONWQHHHLTI-UHFFFAOYNA-N


In [ ]:
import pandas as pd

df = pd.read_csv("/Users/stella/Downloads/tierney/project/combined_data_cleaned.csv")
# Count unique entries
unique_solutes = df['solute_inchikey'].nunique()
unique_solvents = df['solvent_name'].nunique()

print(f"Number of unique solutes InChIKeys: {unique_solutes}")
print(f"Number of unique solvents names: {unique_solvents}")


Number of unique solutes InChIKeys: 3351
Number of unique solvents names: 177
